In [ ]:
import numpy as np
import sympy as syp
import matplotlib.pyplot as plt
from matplotlib import animation


In [10]:

Lx, Ly = 1.0, 1.0        # size
Nx, Ny = 100, 100        # number of grid points
c = 1.0                  # wave speed
T = 5.0                  # total time


x = np.linspace(0, Lx, Nx)
y = np.linspace(0, Ly, Ny)
Dx = x[1] - x[0]
Dy = y[1] - y[0]
Dt = 0.4 * min(Dx, Dy) / c
Nt = int(T / Dt)


# Initialize empty fields
u = np.zeros((Nx, Ny))
u_new = np.zeros((Nx, Ny))
u_old = np.zeros((Nx, Ny))

# Add in motion to center
X, Y = np.meshgrid(x, y, indexing='ij')
u = np.exp(-100 * ((X - Lx/2)**2 + (Y - Ly/2)**2))
u_old = np.copy(u)


# Solve IVBP
frames = []
for i in range(1, Nx-1):
    for j in range(1, Ny-1):
        laplace = ((u[i+1, j] - 2*u[i, j] + u[i-1, j]) / Dx**2 +
                   (u[i, j+1] - 2*u[i, j] + u[i, j-1]) / Dy**2)
        u_new[i, j] = u[i, j] + 0.5 * Dt**2 * c**2 * laplace

frames.append(u.copy())

for n in range(1, Nt):
    for i in range(1, Nx-1):
        for j in range(1, Ny-1):
            laplace = ((u[i+1, j] - 2*u[i, j] + u[i-1, j]) / Dx**2 +
                       (u[i, j+1] - 2*u[i, j] + u[i, j-1]) / Dy**2)
            u_new[i, j] = 2*u[i, j] - u_old[i, j] + Dt**2 * c**2 * laplace

    u_old[:], u[:] = u[:], u_new[:]

    if n % 10 == 0:
        frames.append(u.copy())


In [11]:
# Create video
fig, ax = plt.subplots(figsize=(6, 5))
cax = ax.pcolormesh(X, Y, frames[0], shading='auto', cmap='viridis', vmin=-1, vmax=1)
fig.colorbar(cax)

def animate(i):
    cax.set_array(frames[i].ravel())
    return cax,

ani = animation.FuncAnimation(fig, animate, frames=len(frames), interval=50, blit=True)

ani.save("wave_propagation.mp4", writer="ffmpeg")
plt.close()
